In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import torch
import torch.utils.data
import torch.nn as nn
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from ucimlrepo import fetch_ucirepo
import ssl

# Ignore ssl certificate verification
ssl._create_default_https_context = ssl._create_unverified_context
  
# fetch dataset 
cdc_diabetes_health_indicators = fetch_ucirepo(id=891) 
  
# data (as pandas dataframes) 
X_dataframe = cdc_diabetes_health_indicators.data.features 
y_dataframe = cdc_diabetes_health_indicators.data.targets 
  
# metadata 
print(cdc_diabetes_health_indicators.metadata) 
  
# variable information 
print(cdc_diabetes_health_indicators.variables) 

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

In [3]:
# Converting DataFrames to Pytorch tensors... because who wants to learn Pandas right now?
X = torch.from_numpy(X_dataframe.to_numpy()) 
y = torch.from_numpy(y_dataframe.to_numpy())

print(type(X), type(y))
print(X.shape, y.shape)

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([253680, 21]) torch.Size([253680, 1])


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Converting labels to 1-hot encoding
def to1hot(labels):
    return torch.eye(2)[labels]

labels = to1hot(y[:,0])
print(labels[:3])

# Splitting data into train, validation, and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=.3, stratify=labels, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.5, stratify=y_test, shuffle=True, random_state=42)
print("X_test:", X_test.shape, "X_val:", X_val.shape, "X_train:", X_train.shape)

# Standardizing features
standardizer = StandardScaler()
X_train = torch.from_numpy(standardizer.fit_transform(X_train))
X_val = torch.from_numpy(standardizer.transform(X_val))
X_test = torch.from_numpy(standardizer.transform(X_test))

tensor([[1., 0.],
        [1., 0.],
        [1., 0.]])
X_test: torch.Size([38052, 21]) X_val: torch.Size([38052, 21]) X_train: torch.Size([177576, 21])


In [5]:
from torch.utils.data import DataLoader, Dataset

# Making Datasets for training, validation, and testing
class HealthData(Dataset):
    def __init__(self, participants):
        self.participants = participants

    def __len__(self):
        # The total number of particpants in the dataset
        return len(self.participants)

    def __getitem__(self, idx):
        # Extract the particpant data
        participant = self.participants[idx]
        return participant
    
    
# Each dataset is an R x 23 tensor, where the final 2 columns serve as a 1-hot encoding of the labels
train_dataset = HealthData(torch.concat((X_train, y_train), dim=1).float())
validation_dataset = HealthData(torch.concat((X_val, y_val), dim=1).float())
test_dataset = HealthData(torch.concat((X_test, y_test), dim=1).float())

In [6]:
# Define model with residual connections and implement batch normalization
class BatchNormDNN(nn.Module):
    def __init__(self, input_dim=21, hidden_dim=5, num_classes=2, num_layers=15):
        super(BatchNormDNN, self).__init__()

        # Define layers
        layers = [nn.Linear(input_dim, hidden_dim)]
        for i in range(1, num_layers-1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
        
        layers.append(nn.Linear(hidden_dim, num_classes))
        
        self.layers = nn.ModuleList(layers)
        self.relu = nn.LeakyReLU()
        self.batch_norm = torch.nn.BatchNorm1d(hidden_dim)

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.01)
        
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            if i == 0:
                x = self.relu(self.batch_norm(layer(x)))
            # Don't apply relu or batch_norm to last layer
            elif i == len(self.layers) - 1:
                x =  layer(x)                
            else:
                # Residual connections
                x = x + self.relu(self.batch_norm(layer(x)))
        return x

In [7]:
import torch.optim as optim

def train(model, loss, dataloader, optimizer):
    """Helper function to train our model."""
    total_error = 0.
    for it, batch in enumerate(dataloader):
        inputs = batch[:, :-2]
        labels = batch[:, -2:]
        # Compute model predictions
        pred = model(inputs) 

        optimizer.zero_grad()
        
        # Compute the loss
        l = loss(pred, labels)
        total_error += l.item()
        
        l.backward()

        # Update the weights
        optimizer.step()

    return total_error / len(dataloader)


def fit(model, loss, dataloader, epochs=41):
    optimizer = optim.Adam(model.parameters(), lr=0.03)
    for ep in range(epochs):
        err = train(model, loss, dataloader, optimizer)
        if ep % 10 == 0:
            print(f"[Ep{ep}] Error {err:.3f}")
            
# Define Loss Function
loss = nn.CrossEntropyLoss()

# Define data loader for easy batching
dataloader = DataLoader(train_dataset, batch_size=10000, shuffle=True, drop_last=True)

# Model
model_batch_norm = BatchNormDNN(num_layers=5, hidden_dim=30)
fit(model_batch_norm, loss, dataloader, epochs=50)
    

[Ep0] Error 0.378
[Ep10] Error 0.311
[Ep20] Error 0.310
[Ep30] Error 0.309
[Ep40] Error 0.308


In [8]:
# Test Model
test_model_bn = model_batch_norm

for model in {test_model_bn}:
    test_dataloader = DataLoader(test_dataset, batch_size=5000, shuffle=True, drop_last=True)
    test_error = 0
    test_acc= 0
    print({model})
    for it, batch in enumerate(test_dataloader):
        test_inputs = batch[:, :-2]
        test_labels = batch[:, -2:]
        # Compute model predictions
        test_pred = model(test_inputs)
        l = loss(test_pred, test_labels)
        test_acc += (sum(torch.argmax(test_pred, dim=1) == torch.argmax(test_labels, dim=1)) / len(batch)).item()
        test_error += l.item()
    avg_test_acc = test_acc / len(test_dataloader)    
    avg_error = test_error / len(test_dataloader)
    print("Average Test Accuracy:", avg_test_acc)
    print("Average Test Error:", avg_error)

{BatchNormDNN(
  (layers): ModuleList(
    (0): Linear(in_features=21, out_features=30, bias=True)
    (1-3): 3 x Linear(in_features=30, out_features=30, bias=True)
    (4): Linear(in_features=30, out_features=2, bias=True)
  )
  (relu): LeakyReLU(negative_slope=0.01)
  (batch_norm): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)}
Average Test Accuracy: 0.866399986403329
Average Test Error: 0.3140681173120226


In [9]:
def evaluate_model(model, dataloader):
    all_predictions = []
    all_labels = []

    for batch in dataloader:
        inputs = batch[:, :-2]
        labels = batch[:, -2:]

        # Get model predictions
        predictions = model(inputs)

        # Collect predictions and labels
        all_predictions.extend(torch.argmax(predictions, dim=1).tolist())
        all_labels.extend(torch.argmax(labels, dim=1).tolist())

    # Confusion matrix, Accuracy, Error
    cm = confusion_matrix(all_labels, all_predictions)
    accuracy = accuracy_score(all_labels, all_predictions)
    error = 1 - accuracy
    
    print("Confusion Matrix:")
    print(cm)
    print("Accuracy:", accuracy)
    print("Error:", error)

In [10]:
evaluate_model(model, test_dataloader)

Confusion Matrix:
[[29450   660]
 [ 4051   839]]
Accuracy: 0.8654
Error: 0.13460000000000005
